# Showcasing Protoplast Checkpointing in Perturbation Model

## 1. Introduction

This notebook showcases the checkpointing feature in PROTOplast, which enables resuming model training even after interruptions or switching to a different dataset. It demonstrates how to save and load training checkpoints, making it easy to continue model development without starting from scratch. This is particularly useful for long training sessions, experimentation with various datasets, or training across multiple sessions or environments.

In [1]:
import anndata
import glob
import numpy as np
import pandas as pd
import os
import pathlib
import protoplast as pt
import torch

from anndata.experimental import AnnCollection
from protoplast.scrna.anndata.trainer import RayTrainRunner
from protoplast.scrna.anndata.torch_dataloader import DistributedAnnDataset
from protoplast.scrna.anndata.torch_dataloader import cell_line_metadata_cb

from ray.train import Checkpoint
from ray.train.lightning import RayDDPStrategy

✓ Applied AnnDataFileManager patch


/mnt/hdd2/nam/miniconda3/envs/test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-17 07:12:05,207	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-09-17 07:12:05,334	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-09-17 07:12:05,382	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


✓ Applied AnnDataFileManager patch


## 2. Dataset pre-processing

We begin by reading the two datasets used to train the perturbation model in this notebook. To ensure compatibility, the model requires that both datasets share a common set of features (e.g., genes).

In the following section, we create a unified view by performing an **inner join** on the two datasets based on shared features. During this step, we:

- Identify and record the **number of overlapping genes** (shared features),
- Capture the **indices** of these shared genes in each dataset,
- Extract the list of **perturbed genes** specific to each dataset,
- And prepare **metadata** necessary for consistent training across datasets.

This alignment is essential to ensure the model receives a consistent input/output structure regardless of the dataset source.

In [2]:
DS_PATHS = ["/mnt/hdd2/nam/hct116.h5ad",
           "/mnt/hdd2/tan/competition_support_set/competition_train.h5"]
adatas = [anndata.io.read_h5ad(p, backed = "r") for p in DS_PATHS]

In [3]:
for idx, adata in enumerate(adatas):
    if idx == 0:
        # 
        adata.obs["target_gene"] = adata.obs[["gene_target"]]
        adata.obs["cell_type"] = "HCT116"
    else:
        adata.obs = adata.obs[["target_gene", "cell_type"]]

In [4]:
# Create a view of all dataset
ds_view = AnnCollection(adatas, join_vars = "inner")

# Record the genes shared by the training datasets
n_genes = ds_view.n_vars
genes = ds_view.var_names.tolist()
perts = ds_view.obs["target_gene"].unique().tolist()
cell_types = ds_view.obs["cell_type"].unique().tolist()

print("Number of genes", n_genes)

Number of genes 18080


In [5]:
# Include the indices of the shared variables in the anndata object to help transform
# yieled data batch later in training step
shared_vars = {}
for idx_i, adata_i in enumerate(adatas):
    shared_vars[idx_i] = np.where(np.isin(adata_i.var_names, genes))[0]

## 3. Define model & configure training step

In [6]:
thread_per_worker = 8
test_size = 0.2 
val_size = 0.0 # if you have only training and test data, just put val_size = 0.0

In [9]:
from state.tx.models.perturb_mean import PerturbMeanPerturbationModel # import original class

class PerturbMeanGlobalModel(PerturbMeanPerturbationModel):
    def __init__(self, *wargs, **kwargs):
        kwargs['gene_decoder_bool'] = False
        super().__init__(*wargs, **kwargs)

    """
    Extended class of PerturbMeanPerturbationModel where prediction ignores
    per-cell control embedding and uses only global basal + offset.
    """

    def forward(self, batch: dict) -> torch.Tensor:
        B = len(batch["pert_name"])
        device = self.dummy_param.device
        pred_out = torch.zeros((B, self.output_dim), device=device)

        for i in range(B):
            p_name = str(batch["pert_name"][i])
            offset_vec = self.pert_mean_offsets.get(p_name)
            if offset_vec is None:
                offset_vec = torch.zeros(self.output_dim, device=device)

            # Use global basal instead of batch["ctrl_cell_emb"]
            pred_out[i] = self.global_basal.to(device) + offset_vec.to(device)

        return pred_out

## 4. Train on `HCT116_filtered_dual_guide_cells` dataset

We first train this perturbation model on a dataset that contains HCT116 cells. We need to define a callback function to set up a metadata for the perturbation model based on the dataset's attributes.

In [7]:
hct116_adata = anndata.read_h5ad(DS_PATHS[0], backed = "r")

In [8]:
hct116_adata.obs.head(n = 5)

,sample,num_features,guide_target,gene_target,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,pass_guide_filter,target_gene,cell_type,batch_var
AAACCAAAGACATGTT-HCT116_Batch1,HCT116_Batch1,2,ST14_P1P2-1|ST14_P1P2-2,ST14,4883,19136.0,1179.0,6.161162,True,ST14,hct116,HCT116_Batch1
AAACCAAAGACCCAAC-HCT116_Batch1,HCT116_Batch1,2,SIGLEC5_P1P2-1|SIGLEC5_P1P2-2,SIGLEC5,8130,47916.0,1562.0,3.259871,True,SIGLEC5,hct116,HCT116_Batch1
AAACCAAAGAGGTACG-HCT116_Batch1,HCT116_Batch1,2,VSNL1_P1P2-1|VSNL1_P1P2-2,VSNL1,6531,28435.0,1042.0,3.664498,True,VSNL1,hct116,HCT116_Batch1
AAACCAAAGCGATTAT-HCT116_Batch1,HCT116_Batch1,2,KCNK7_P1P2-1|KCNK7_P1P2-2,KCNK7,5931,26080.0,1087.0,4.167945,True,KCNK7,hct116,HCT116_Batch1
AAACCAAAGGCTTAAT-HCT116_Batch1,HCT116_Batch1,2,APOA4_P1P2-1|APOA4_P1P2-2,APOA4,7157,38366.0,955.0,2.489183,True,APOA4,hct116,HCT116_Batch1


In [9]:
def hct116_perturbmean_metadata_cb(ad: anndata.AnnData, metadata: dict):
    ad.obs["gene_target"] = ad.obs["gene_target"].astype("category")
    ad.obs["cell_type"] = "HCT116"

    metadata["input_dim"] = n_genes
    metadata["output_dim"] = n_genes
    metadata["hidden_dim"] = 10
    metadata["pert_dim"] = len(perts)
    metadata["lr"] = 1e-3  

    metadata["gene_names"] = genes
    metadata["pert_names"] = perts
    metadata["cell_types"] = cell_types

    metadata["control_pert"] = "Non-Targeting"
    metadata["embed_key"] = "X"
    metadata["output_space"] = "gene"

In [10]:
class HCT116_PerturbAnnDataset(DistributedAnnDataset):
    def transform(self, start: int, end: int):
        X = super().transform(start, end)

        # Subset X matrix to include only genes appear in all dataset
        # Need to densify the data
        X = X.to_dense()[:, shared_vars[0]]

        # Metadata froms self.ad
        pert_names = self.ad.obs["gene_target"].iloc[start:end].astype(str).to_list()
        cell_lines = ["HCT116"] * len(pert_names)

        return {
            "pert_name": pert_names,
            "cell_type": cell_lines,
            "pert_cell_counts": X,
            "pert_cell_emb": X,
        }

In [12]:
# Set up training
PerturbMeanPerturbationModel_trainer = RayTrainRunner(
    PerturbMeanGlobalModel,
    HCT116_PerturbAnnDataset,
    ["input_dim",
    "output_dim",
    "hidden_dim",      
    "pert_dim",        
    "lr",
    "control_pert",    # "Non-Tageting"
    "embed_key",       
    "output_space",    # "gene"
    ],
    metadata_cb = hct116_perturbmean_metadata_cb,
)

2025-09-17 07:10:31,573	INFO worker.py:1951 -- Started a local Ray instance.


(TrainTrainable pid=313216) ✓ Applied AnnDataFileManager patch
(TrainTrainable pid=313216) ✓ Applied AnnDataFileManager patch


(RayTrainWorker pid=313514) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=313216) Started distributed worker processes: 
(TorchTrainer pid=313216) - (node_id=e17c683c1f9c773c17fac7886b21de1ee3c757be3765d231a371137b, ip=192.168.1.226, pid=313514) world_rank=0, local_rank=0, node_rank=0


(RayTrainWorker pid=313514) ✓ Applied AnnDataFileManager patch
(RayTrainWorker pid=313514) ✓ Applied AnnDataFileManager patch


(RayTrainWorker pid=313514) 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
(RayTrainWorker pid=313514) GPU available: True (cuda), used: True
(RayTrainWorker pid=313514) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=313514) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=313514) You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


...

(RayTrainWorker pid=313514) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/nam/protoplast_results/TorchTrainer_2025-09-17_07-10-35/TorchTrainer_6851c_00000_0_2025-09-17_07-10-35/checkpoint_000000)


Epoch 0: 100%|██████████| 126/126 [00:09<00:00, 12.94it/s, v_num=0, train_loss=0.0197]


(RayTrainWorker pid=313514) `Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 126/126 [00:14<00:00,  8.63it/s, v_num=0, train_loss=0.0197]


(RayTrainWorker pid=313514) [rank0]:[W917 07:11:21.595333013 ProcessGroupNCCL.cpp:1538] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


In [13]:
PerturbMeanPerturbationModel_trainer.train([DS_PATHS[0]],
                                           thread_per_worker = thread_per_worker, 
                                           batch_size = 64,
                                           test_size = test_size, 
                                           val_size = val_size,
                                           num_workers = 1,
                                           resource_per_worker = {"GPU": 1})

Using 1 workers with {'GPU': 1} each
Data splitting time: 0.09 seconds
Spawning Ray worker and initiating distributed training


2025-09-17 07:10:35,780	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2025-09-17 07:10:36 (running for 00:00:00.18)
Using FIFO scheduling algorithm.
Logical resource usage: 0/96 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-09-17_07-10-27_487051_304131/artifacts/2025-09-17_07-10-35/TorchTrainer_2025-09-17_07-10-35/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-09-17 07:10:41 (running for 00:00:05.21)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-09-17_07-10-27_487051_304131/artifacts/2025-09-17_07-10-35/TorchTrainer_2025-09-17_07-10-35/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-09-17 07:10:46 (running for 00:00:10.26)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-09-17_07-10-27_487051_304131/artifacts/2025-09-17_07-1

2025-09-17 07:11:19,834	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/nam/protoplast_results/TorchTrainer_2025-09-17_07-10-35' in 0.0463s.
2025-09-17 07:11:19,838	INFO tune.py:1041 -- Total run time: 44.06 seconds (43.96 seconds for the tuning loop).


== Status ==
Current time: 2025-09-17 07:11:19 (running for 00:00:44.01)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-09-17_07-10-27_487051_304131/artifacts/2025-09-17_07-10-35/TorchTrainer_2025-09-17_07-10-35/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)




Result(
  metrics={'train_loss': 0.019687622785568237, 'epoch': 0, 'step': 126},
  path='/home/nam/protoplast_results/TorchTrainer_2025-09-17_07-10-35/TorchTrainer_6851c_00000_0_2025-09-17_07-10-35',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/home/nam/protoplast_results/TorchTrainer_2025-09-17_07-10-35/TorchTrainer_6851c_00000_0_2025-09-17_07-10-35/checkpoint_000000)
)

## 5. Train on `competition_train` dataset

We now have a checkpoint saved after training the perturbation model using the first dataset. To continue training on a different dataset, several adjustments are necessary to ensure compatibility and correct model behavior.

- We need to define a new **metadata callback function** that sets up the appropriate configurations for the model when training under the new dataset.

Since the second dataset may have different **input dimensions** or **metadata fields**, we also define a custom `AnnDataset` class. This class is responsible for transforming each training batch accordingly, ensuring:

- Features are mapped to the expected input space,
- Metadata is correctly aligned with the model's expectations,
- Any dataset-specific preprocessing is applied consistently.

In [10]:
competition_adata = anndata.read_h5ad(DS_PATHS[1], backed = "r")

In [11]:
competition_adata.obs.head(n = 5)

,target_gene,guide_id,batch,batch_var,cell_type
AAACAAGCAACCTTGTACTTTAGG-Flex_1_01,CHMP3,CHMP3_P1P2_A|CHMP3_P1P2_B,Flex_1_01,Flex_1_01,ARC_H1
AAACAAGCATTGCCGCACTTTAGG-Flex_1_01,AKT2,AKT2_P1P2_A|AKT2_P1P2_B,Flex_1_01,Flex_1_01,ARC_H1
AAACCAATCAATGTTCACTTTAGG-Flex_1_01,SHPRH,SHPRH_P1P2_A|SHPRH_P1P2_B,Flex_1_01,Flex_1_01,ARC_H1
AAACCAATCCCTCGCTACTTTAGG-Flex_1_01,TMSB4X,TMSB4X_P1_A|TMSB4X_P1_B,Flex_1_01,Flex_1_01,ARC_H1
AAACCAATCTAAATCCACTTTAGG-Flex_1_01,KLF10,KLF10_P2_A|KLF10_P2_B,Flex_1_01,Flex_1_01,ARC_H1


In [12]:
def competition_perturbmean_metadata_cb(ad: anndata.AnnData, metadata: dict):
    metadata["input_dim"] = n_genes
    metadata["output_dim"] = n_genes
    metadata["hidden_dim"] = 10
    metadata["pert_dim"] = len(perts)
    metadata["lr"] = 1e-3

    metadata["gene_names"] = genes
    metadata["pert_names"] = perts
    metadata["cell_types"] = cell_types

    metadata["control_pert"] = "non-targeting"
    metadata["embed_key"] = "X"
    metadata["output_space"] = "gene"

In [13]:
class Competition_PerturbAnnDataset(DistributedAnnDataset):
    def transform(self, start: int, end: int):
        X = super().transform(start, end)

        # Subset X matrix to include only genes appear in all dataset
        # Need to densify the data
        X = X.to_dense()[:, shared_vars[1]]

        # Metadata froms self.ad
        pert_names = self.ad.obs["target_gene"].iloc[start:end].astype(str).to_list()
        cell_lines = self.ad.obs["cell_type"].iloc[start:end].astype(str).to_list()

        return {
            "pert_name": pert_names,
            "cell_type": cell_lines,
            "pert_cell_counts": X,
            "pert_cell_emb": X,
        }

In [14]:
# Set up training
competition_trainer = RayTrainRunner(
    PerturbMeanGlobalModel,
    Competition_PerturbAnnDataset,
    ["input_dim",
    "output_dim",
    "hidden_dim",      
    "pert_dim",
    "lr",
    "control_pert",    # "non-targeting"
    "embed_key",       
    "output_space",    # "gene"
    ],
    metadata_cb = competition_perturbmean_metadata_cb,
)

2025-09-17 07:12:47,019	INFO worker.py:1951 -- Started a local Ray instance.


(TrainTrainable pid=322902) ✓ Applied AnnDataFileManager patch
(TrainTrainable pid=322902) ✓ Applied AnnDataFileManager patch


(RayTrainWorker pid=323141) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=322902) Started distributed worker processes: 
(TorchTrainer pid=322902) - (node_id=7ec1987a6e39c9f76fbf76f3a5a37172cc22e2732324dfe61cc518c1, ip=192.168.1.226, pid=323141) world_rank=0, local_rank=0, node_rank=0


(RayTrainWorker pid=323141) ✓ Applied AnnDataFileManager patch
(RayTrainWorker pid=323141) ✓ Applied AnnDataFileManager patch


(RayTrainWorker pid=323141) 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
(RayTrainWorker pid=323141) GPU available: True (cuda), used: True
(RayTrainWorker pid=323141) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=323141) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=323141) You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


...

(RayTrainWorker pid=323141) Restoring states from the checkpoint path at /home/nam/protoplast_results/TorchTrainer_2025-09-17_07-10-35/TorchTrainer_6851c_00000_0_2025-09-17_07-10-35/checkpoint_000000/checkpoint.ckpt
(RayTrainWorker pid=323141) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
(RayTrainWorker pid=323141) 
(RayTrainWorker pid=323141)   | Name         | Type    | Params | Mode 
(RayTrainWorker pid=323141) -------------------------------------------------
(RayTrainWorker pid=323141) 0 | loss_fn      | MSELoss | 0      | train
(RayTrainWorker pid=323141)   | other params | n/a     | 1      | n/a  
(RayTrainWorker pid=323141) -------------------------------------------------
(RayTrainWorker pid=323141) 1         Trainable params
(RayTrainWorker pid=323141) 0         Non-trainable params
(RayTrainWorker pid=323141) 1         Total params
(RayTrainWorker pid=323141) 0.000     Total estimated model params size (MB)
(RayTrainWorker pid=323141) 1         Modules in train mode
(RayTrainWo

                                                  
Epoch 1: 100%|██████████| 88/88 [02:16<00:00,  0.64it/s, v_num=0, train_loss=0.179]


(RayTrainWorker pid=323141) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/nam/protoplast_results/TorchTrainer_2025-09-17_07-12-53/TorchTrainer_ba6c4_00000_0_2025-09-17_07-12-53/checkpoint_000000)
(RayTrainWorker pid=323141) `Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 88/88 [02:17<00:00,  0.64it/s, v_num=0, train_loss=0.179]


(RayTrainWorker pid=323141) [rank0]:[W917 07:17:57.849372713 ProcessGroupNCCL.cpp:1538] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


In [15]:
ckpt_path = ("/home/nam/protoplast_results/TorchTrainer_2025-09-17_07-10-35/TorchTrainer_6851c_00000_0_2025-09-17_07-10-35/checkpoint_000000/checkpoint.ckpt")

competition_trainer.train([DS_PATHS[1]],
                          max_epochs = 2,
                          thread_per_worker = thread_per_worker, 
                          batch_size = 2048, 
                          test_size = test_size, 
                          val_size = val_size,
                          num_workers = 1,
                          resource_per_worker = {"GPU": 1},
                          ckpt_path = ckpt_path)

Using 1 workers with {'GPU': 1} each


2025-09-17 07:12:53,540	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Data splitting time: 0.32 seconds
Spawning Ray worker and initiating distributed training
== Status ==
Current time: 2025-09-17 07:12:53 (running for 00:00:00.18)
Using FIFO scheduling algorithm.
Logical resource usage: 0/96 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-09-17_07-12-42_953619_316222/artifacts/2025-09-17_07-12-53/TorchTrainer_2025-09-17_07-12-53/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-09-17 07:12:58 (running for 00:00:05.21)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-09-17_07-12-42_953619_316222/artifacts/2025-09-17_07-12-53/TorchTrainer_2025-09-17_07-12-53/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-09-17 07:13:03 (running for 00:00:10.26)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
R

2025-09-17 07:17:55,815	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/nam/protoplast_results/TorchTrainer_2025-09-17_07-12-53' in 0.0611s.
2025-09-17 07:17:55,818	INFO tune.py:1041 -- Total run time: 302.28 seconds (302.18 seconds for the tuning loop).


== Status ==
Current time: 2025-09-17 07:17:55 (running for 00:05:02.24)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-09-17_07-12-42_953619_316222/artifacts/2025-09-17_07-12-53/TorchTrainer_2025-09-17_07-12-53/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)




Result(
  metrics={'train_loss': 0.1792626529932022, 'epoch': 1, 'step': 214},
  path='/home/nam/protoplast_results/TorchTrainer_2025-09-17_07-12-53/TorchTrainer_ba6c4_00000_0_2025-09-17_07-12-53',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/home/nam/protoplast_results/TorchTrainer_2025-09-17_07-12-53/TorchTrainer_ba6c4_00000_0_2025-09-17_07-12-53/checkpoint_000000)
)

### Conclusion

This brings us to the end of the tutorial notebook.

This workflow highlights using checkpointing in **PROTOplast**, enabling efficient model development across diverse datasets.

Feel free to explore and extend this notebook to suit your own data and use cases!